In [1]:
import pandas as pd


In [2]:
data = pd.read_csv("CP_count_Hofmarcher.csv")
data

,INCHIKEY,1,2,3,4,5,6,7,8,9,...,Cells_Neighbors_SecondClosestObjectNumber_5,Cells_Neighbors_SecondClosestObjectNumber_Adjacent,Cells_Parent_Nuclei,Cytoplasm_Number_Object_Number,Cytoplasm_Parent_Cells,Cytoplasm_Parent_Nuclei,Nuclei_Neighbors_FirstClosestObjectNumber_1,Nuclei_Neighbors_SecondClosestObjectNumber_1,Nuclei_Number_Object_Number,InChIKey
0,AACRWZVDRSTLKY-UHFFFAOYSA-N,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.054688,-0.054688,0.859375,0.859375,0.859375,0.859375,1.445312,1.007812,0.859375,AACRWZVDRSTLKY-UHFFFAOYSA-N
1,AACUKVXTFOXDGE-UHFFFAOYSA-N,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.171875,-1.171875,-1.148438,-1.148438,-1.148438,-1.148438,-1.101562,-1.453125,-1.148438,AACUKVXTFOXDGE-UHFFFAOYSA-N
2,AADCDMQTJNYOSS-LBPRGKRZSA-N,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.425781,-0.425781,0.453125,0.453125,0.453125,0.453125,-0.117188,-0.531250,0.453125,AADCDMQTJNYOSS-LBPRGKRZSA-N
3,AADORYZVGJDNSZ-UHFFFAOYSA-N,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1.123188,1.123188,0.521739,0.521739,0.521739,0.521739,1.536232,0.601449,0.521739,AADORYZVGJDNSZ-UHFFFAOYSA-N
4,AAEVYOVXGOFMJO-UHFFFAOYSA-N,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-6.882812,-6.882812,-7.218750,-7.218750,-7.218750,-7.218750,-7.679688,-7.773438,-7.218750,AAEVYOVXGOFMJO-UHFFFAOYSA-N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10568,ZZUCJGSOKDNIEZ-UHFFFAOYSA-N,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0.609375,0.609375,0.679688,0.679688,0.679688,0.679688,0.757812,0.757812,0.679688,ZZUCJGSOKDNIEZ-UHFFFAOYSA-N
10569,ZZUFCTLCJUWOSV-UHFFFAOYSA-N,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-10.242188,-10.242188,-10.746094,-10.746094,-10.746094,-10.746094,-11.167969,-10.281250,-10.746094,ZZUFCTLCJUWOSV-UHFFFAOYSA-N
10570,ZZUZYEMRHCMVTB-UHFFFAOYSA-N,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-16.171875,-16.171875,-16.304688,-16.304688,-16.304688,-16.304688,-16.335938,-15.152344,-16.304688,ZZUZYEMRHCMVTB-UHFFFAOYSA-N
10571,ZZVUWRFHKOJYTH-UHFFFAOYSA-N,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.640625,-0.640625,-1.664062,-1.664062,-1.664062,-1.664062,-1.257812,-0.289062,-1.664062,ZZVUWRFHKOJYTH-UHFFFAOYSA-N


In [3]:
assays_list = data.columns.to_list()[1:-13]
assays_list

train_cols = ['Cells_Number_Object_Number']

In [4]:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import logging


cp_path = "CP_count_Hofmarcher.csv"
cp_data = pd.read_csv(cp_path)

def merge_data(split_data, cp_data):
    # Assuming 'inchikey' is the column to merge on
    return pd.merge(split_data, cp_data, on='INCHIKEY', how='inner')

def load_and_preprocess_data(fold):
    train_path = f"data/datasplit{fold}-train.csv"
    val_path = f"data/datasplit{fold}-val.csv"
    test_path = f"data/datasplit{fold}-test.csv"

    # Load CP data
    cp_data = pd.read_csv(cp_path)
    
    # Load data splits
    train_df= pd.read_csv(train_path)
    val_df = pd.read_csv(val_path)
    test_df = pd.read_csv(test_path)
    
    # Merge the datasets
    train_merged = merge_data(train_df, cp_data)
    val_merged = merge_data(val_df, cp_data)
    test = merge_data(test_df, cp_data)
    
    train = pd.concat([train_merged,val_merged]).reset_index(drop=True)
    
    return train, test



In [14]:
import numpy as np
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.utils import class_weight
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold

missing_value_indicator = -1

# Initialize a list to store AUC scores for all folds
all_auc_scores = []

# Initialize an empty list to hold models for each fold
models_per_fold = []

# Define the fold indices (adjust based on your data split)
folds = ["1", "2", "3"]

best_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42, n_jobs=-1)

# Loop through each fold, leaving it out as the test set and training on the others
for fold in tqdm(folds):
    
    if(best_clf):      
        del(best_clf)

    #Train on all folds except the current one
    train_data, test_data = load_and_preprocess_data(fold)

    # Loop through all assays and collect data
    for assay in tqdm(assays_list):
        
        if(best_clf):      
        del(best_clf)
        
        train_data_assay = train_data[train_data[assay]!=-1].reset_index(drop=True)[[assay, "Cells_Number_Object_Number"]]

        # Append to combined training data
        X_train = train_data_assay[["Cells_Number_Object_Number"]]
        y_train = train_data_assay[assay].astype(int)

        test_data_assay = test_data[test_data[assay]!=-1].reset_index(drop=True)[[assay, "Cells_Number_Object_Number"]]
        
        # Separate features and target variable
        X_test = test_data_assay[["Cells_Number_Object_Number"]]
        y_test = test_data_assay[assay].astype(int)

        # print(len(y_train))
        # print(len(y_test))
        
        if len(y_test) == 0:
            print(f"Skipping assay {assay} in fold {fold} due to no test data")
            continue
          
        # Calculate class weights (balanced)
        class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
        class_weight_dict = {0: class_weights[0], 1: class_weights[1]}
        # print(class_weight_dict)

        # Calculate scale_pos_weight based on class weights
        scale_pos_weight = class_weight_dict[0] / class_weight_dict[1]
        # print("scale_pos_weight: ", scale_pos_weight)

        # Define the parameter grid with additional hyperparameters and wider ranges
        param_grid = {
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1, 0.3],
        'n_estimators': [100, 200],
        'scale_pos_weight': [scale_pos_weight, 1],
        'n_jobs': [-1]
        }

        # Initialize the XGBoost classifier
        xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42, n_jobs=-1)

        stratified_kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

        # Perform RandomizedSearchCV
        random_search = RandomizedSearchCV(
            estimator=xgb_clf,
            param_distributions=param_grid,
            n_iter=10,  # Number of parameter settings that are sampled
            scoring='f1',  # Metric for imbalanced dataset (you can also try 'roc_auc' or 'average_precision')
            cv=stratified_kfold,  # 5-fold cross-validation
            n_jobs=-1,  # Use all processors
            verbose=0,
            random_state=42
        )

        # Fit the model on training data
        random_search.fit(X_train, y_train)

        # Best estimator
        best_clf = random_search.best_estimator_
        # print(f"Best parameters: {random_search.best_params_}")

        # Train the model on the combined data from other folds
        # clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42, n_jobs=-1)
        best_clf.fit(X_train, y_train)

        # Make predictions using the model for the current fold
        y_pred = best_clf.predict(X_test)
        y_pred_proba = best_clf.predict_proba(X_test)[:, 1]

        # Calculate AUC score
        auc_score = np.nan    
        try:
            auc_score = roc_auc_score(y_test, y_pred_proba)
        except:
            continue
        # print(auc_score)

        # Store the AUC score along with fold and assay information
        all_auc_scores.append({"Fold": fold, "Task": assay, "AUC": auc_score})

# Convert the AUC results to a DataFrame for convenience
auc_df = pd.DataFrame(all_auc_scores)

# Save the DataFrame to a CSV file
auc_df.to_csv("auc_scores_across_folds_individualmodels.csv", index=False)

# Display the AUC DataFrame
auc_df


  0%|          | 0/3 [00:00<?, ?it/s]/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:34: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.

 33%|███▎      | 1/3 [03:32<07:04, 212.09s/it]/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:34: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.

 67%|██████▋   | 2/3 [08:51<04:35, 275.44s/it]/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:34: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.

100%|██████████| 3/3 [13:08<00:00, 262.93s/it]


,Fold,Task,AUC
0,1,1,0.555556
1,1,2,0.489796
2,1,4,0.415385
3,1,5,0.250000
4,1,6,0.505435
...,...,...,...
613,3,205,0.555398
614,3,206,0.305556
615,3,207,0.928571
616,3,208,0.764706


In [30]:
mean_auc = auc_df.groupby("Task").mean().reset_index(drop=False)
mean_auc

,Task,AUC
0,1,0.550831
1,10,0.445697
2,100,0.910435
3,101,0.972066
4,102,0.853547
...,...,...
204,95,0.901316
205,96,0.888889
206,97,0.916667
207,98,0.912464


In [31]:
len(mean_auc[mean_auc["AUC"]==1])


0

In [32]:
len(mean_auc[mean_auc["AUC"]>0.90])

57

In [33]:
len(mean_auc[mean_auc["AUC"]>0.80])

73

In [34]:
len(mean_auc[mean_auc["AUC"]>0.70])

82

In [35]:
len(mean_auc[mean_auc["AUC"]>0.60])

118

In [36]:
len(mean_auc[mean_auc["AUC"]>0.50])

167

In [37]:
mean_auc.mean()

Task         inf
AUC     0.676478
dtype: float64

In [38]:
mean_auc.std()

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


AUC    0.207838
dtype: float64

In [39]:
mean_auc.to_csv("scaffold_mean_AUCSeal_individualmodels.csv")

In [40]:
median_auc = auc_df.groupby("Task").median().reset_index(drop=False)
median_auc.to_csv("scaffold_median_AUCSeal_individualmodels.csv")

In [41]:
len(median_auc[median_auc["AUC"]==1])

2

In [42]:
len(median_auc[median_auc["AUC"]>0.90])

62

In [43]:
len(median_auc[median_auc["AUC"]>0.70])

91

In [44]:
len(median_auc[median_auc["AUC"]>0.50])

160